In [1]:
from pynq_dpu import DpuOverlay
from uartlite import UartLite
overlay = DpuOverlay('dpu_uartlite.bit')
uart = UartLite(overlay.axi_uartlite_0)

print("UART overlay done")

UART overlay done


In [34]:
uart.write("02551255\n")

9

In [29]:
uart.write("01270127\n")

9

In [40]:
uart.write("10001000\n")

9

In [82]:
def cmd_vel_control(linear_x, angular_z):
    WHEEL_DIST = 5;
    speed_wish_right = (angular_z * WHEEL_DIST)/2 + linear_x
    speed_wish_left = linear_x * 2-speed_wish_right
    
    speed_wish_right = int(255*speed_wish_right)
    speed_wish_left = int(255*speed_wish_left)
    if speed_wish_right > 255: speed_wish_right = 255
    if speed_wish_right < -255: speed_wish_right = -255
    if speed_wish_left > 255: speed_wish_left = 255
    if speed_wish_left < -255: speed_wish_left = -255
    print("speed_wish_right, speed_wish_left= ",speed_wish_right, ",",speed_wish_left)
    
    
    pkg="1" if speed_wish_right>=0 else "0"
    speed_wish_right = abs(speed_wish_right)
    pkg += str(speed_wish_right//100)
    pkg += str((speed_wish_right%100)//10)
    pkg += str(speed_wish_right%10)
    
    if speed_wish_left>=0: pkg += "1"
    else : pkg += "0"
    speed_wish_left = abs(speed_wish_left)
    pkg += str(speed_wish_left//100)
    pkg += str((speed_wish_left%100)//10)
    pkg += str(speed_wish_left%10)
    pkg += "\n"
    
    print("uart pkg= ", pkg)
    
    uart.write(pkg)

In [93]:
cmd_vel_control(0, 0)

speed_wish_right, speed_wish_left=  0 , 0
uart pkg=  10001000



In [91]:
cmd_vel_control(0.5, 0)

speed_wish_right, speed_wish_left=  127 , 127
uart pkg=  11271127



In [92]:
cmd_vel_control(0, -0.5)

speed_wish_right, speed_wish_left=  -255 , 255
uart pkg=  02551255

